# Strategy 1 with Optimizations

## Import Modules

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
import xlsxwriter
import openpyxl
from statistics import mean

## Import and Clean Data 

In [2]:
nse_data = pd.read_excel("Bank Nifty data 5 min.xlsx", header = 2)

In [3]:
nse_data.head()

,Date,Time,Open,High,Low,Close,SMA20,SMA50,SMA 100
0,2011-07-29,09:15:00,10820.65,10820.65,10758.40,10774.20,NaN,NaN,NaN
1,2011-07-29,09:20:00,10774.20,10812.40,10774.20,10807.95,NaN,NaN,NaN
2,2011-07-29,09:25:00,10807.95,10837.25,10804.35,10837.25,NaN,NaN,NaN
3,2011-07-29,09:30:00,10836.05,10891.45,10835.60,10884.80,NaN,NaN,NaN
4,2011-07-29,09:35:00,10884.80,10892.30,10871.85,10891.65,NaN,NaN,NaN


In [4]:
nse_data = nse_data.dropna()
nse_data.reset_index(drop=True, inplace = True)

In [5]:
nse_data.head()

,Date,Time,Open,High,Low,Close,SMA20,SMA50,SMA 100
0,2011-08-01,0.472222,10994.30,10995.10,10991.00,10992.45,11000.5125,10958.851,10901.713366
1,2011-08-01,0.475694,10992.45,10996.60,10991.40,10991.75,10997.7750,10962.729,10903.867327
2,2011-08-01,0.479167,10991.75,11012.95,10988.80,11012.15,10996.9250,10966.335,10905.889109
3,2011-08-01,0.482639,11010.20,11013.45,11003.65,11006.00,10996.0650,10968.798,10907.559901
4,2011-08-01,0.486111,11006.00,11008.50,11001.55,11008.50,10995.6200,10971.343,10908.784653


In [6]:
nse_data.tail()

,Date,Time,Open,High,Low,Close,SMA20,SMA50,SMA 100
192228,2022-01-24,15:05:00,36874.30,36946.70,36813.45,36946.70,36725.6400,36968.661,37242.978218
192229,2022-01-24,15:10:00,36946.55,37063.65,36929.50,37038.35,36735.3350,36960.372,37237.150000
192230,2022-01-24,15:15:00,37048.60,37073.95,36990.90,37050.30,36747.5000,36953.933,37231.681188
192231,2022-01-24,15:20:00,37047.80,37062.85,36894.25,36905.65,36751.2025,36944.372,37223.801980
192232,2022-01-24,15:25:00,36902.40,36945.05,36808.15,36834.90,36751.3125,36934.724,37215.664356


In [7]:
nse_data.describe()

,Open,High,Low,Close,SMA20,SMA50,SMA 100
count,192233.000000,192233.000000,192233.000000,192233.000000,192233.000000,192233.000000,192233.000000
mean,20702.407157,20719.034539,20685.004447,20702.128129,20700.852455,20698.833568,20695.336316
std,8218.150657,8223.917602,8211.927866,8217.985252,8217.216728,8216.001817,8213.814845
min,7772.600000,7801.950000,7766.350000,7772.600000,7828.862500,7836.277000,7855.661881
25%,12718.750000,12726.700000,12710.650000,12718.650000,12717.780000,12713.638000,12705.542574
50%,19203.150000,19219.500000,19184.750000,19201.950000,19203.905000,19200.594000,19193.671287
75%,26865.200000,26879.700000,26847.950000,26864.550000,26866.905000,26872.199000,26874.929208
max,41729.900000,41827.650000,41600.550000,41722.200000,41556.880000,41367.579000,41231.838614


## Define Functions 

In [8]:
def plot_returns(x, y, xlabel, ylabel, title):
    plt.plot(x, y)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.title(title)
    plt.show()

In [26]:
def find_drawdown(returns_list):
    max_drawDown = 0
    drawDown_percent = "NA"
    index = 0
    while index < len(returns_list):
        positive_sum = 0
        negative_sum = 0

        if returns_list[index] > 0:
            while index < len(returns_list) and returns_list[index] > 0:
                positive_sum += returns_list[index]
                index += 1
        if index < len(returns_list) and returns_list[index] < 0:
            while index < len(returns_list) and returns_list[index] < 0:
                negative_sum += returns_list[index]
                index += 1

        if negative_sum < max_drawDown: 
            max_drawDown = negative_sum
            max_dd_peak = positive_sum
            drawDown_percent = round(negative_sum/max_dd_peak*100,2) if max_dd_peak != 0 else "NA"

    return max_drawDown, drawDown_percent

In [27]:
def find_drawup(returns_list):
    max_drawUp = 0
    drawUp_percent = "NA"
    index = 0
    while index < len(returns_list):
        positive_sum = 0
        negative_sum = 0

        if returns_list[index] < 0:
            while index < len(returns_list) and returns_list[index] < 0:
                negative_sum += returns_list[index]
                index += 1
        if index < len(returns_list) and returns_list[index] > 0:
            while index < len(returns_list) and returns_list[index] > 0:
                positive_sum += returns_list[index]
                index += 1

        if positive_sum > max_drawUp: 
            max_drawUp = positive_sum
            max_du_through = negative_sum
            drawUp_percent = round(positive_sum/(-max_du_through)*100,2) if max_du_through != 0 else "NA"

    return max_drawUp, drawUp_percent

In [64]:
def fill_excel(returns_list, year, all_close_values, df, wb):
    if len(all_close_values) != 2*len(returns_list):
        print("\nCHECK THE NUMBER OF CLOSE VALUES AND THE NUMBER OF RETURNS\n")

    win_trades = list(filter(lambda x: (x > 0), returns_list))
    loose_trades = list(filter(lambda x: (x < 0), returns_list))

    try:
        ws = wb[str(year)]
    except:
        wb.create_sheet(str(year))
        ws = wb[str(year)]
        ws.title = str(year)
    ws.column_dimensions['F'].width = 30
    ws.column_dimensions['G'].width = 8
    ws.column_dimensions['H'].width = 30

    avg_index_val = round(mean(all_close_values),2) if len(all_close_values) != 0 else "NA" 
    drawUp, drawUp_percentage = find_drawup(returns_list)
    drawDown, drawDown_percentage = find_drawdown(returns_list)
    max_return = max(win_trades) if len(win_trades) != 0 else "NA"
    min_return = min(loose_trades) if len(loose_trades) != 0 else "NA"
    if max_return != "NA" and min_return != "NA":
        risk_reward = round(max_return/(-min_return),2)
    else:
        risk_reward = "NA"

    parameters = {
        "Averaeg Index Value": avg_index_val,
        "Total Trades": len(returns_list),
        "Total Win Trades": len(win_trades),
        "Total Loose Trades": len(loose_trades),
        "Hit Ratio or Trade win Ratio": round(len(win_trades)/len(returns_list)*100,2) if len(returns_list) != 0 else "NA",
        "Day win Ratio": round(len(win_trades)/len(loose_trades)*100,2) if len(loose_trades) != 0 else "NA",
        "Gross Profit": sum(win_trades),
        "Gross Loss": sum(loose_trades),
        "Net Profit": sum(win_trades)+sum(loose_trades),
        "Max Draw Up": drawUp,
        "Max Draw Down": drawDown,
        "Draw UP Percentage": drawUp_percentage,
        "Draw Down Percentage": drawDown_percentage,
        "Average Trades": round((sum(win_trades)+sum(loose_trades))/len(returns_list),2) if len(returns_list) != 0 else "NA",
        "Average Winning Trade": round(mean(win_trades),2) if len(win_trades) != 0 else "NA",
        "Average Loosing Trade": round(mean(loose_trades),2) if len(loose_trades) != 0 else "NA",
        "Max Return/ Largest Winning Trade": max_return,
        "Min Return/ Largest Loosing Trade": min_return,
        "Risk Reward": risk_reward,
        "Net Profit %": round((sum(win_trades)+sum(loose_trades))/avg_index_val*100,2) if avg_index_val != "NA" else "NA",
        "Sharpe Ratio": "???"
    }

    definitions = {
        "Averaeg Index Value": "Average(Close Values)",
        "Total Trades": "",
        "Total Win Trades":"",
        "Total Loose Trades": "",
        "Hit Ratio or Trade win Ratio": "win_trades/total_trades*100",
        "Day win Ratio": "(win_trades)/(loose_trades)*100",
        "Gross Profit": "sum(win_trades)",
        "Gross Loss": "sum(loose_trades)",
        "Net Profit": "sum(win_trades)+sum(loose_trades)",
        "Max Draw Up": "Highest Jump from Trough to Peak",
        "Max Draw Down": "Highest Jump from Peak to Through",
        "Draw UP Percentage": "Max Draw Up/ Sum of loss",
        "Draw Down Percentage": "Max Draw Down / Sum of Profit",
        "Average Trades": "sum(win_trades)+sum(loose_trades))/total_trades",
        "Average Winning Trade": "",
        "Average Loosing Trade": "",
        "Max Return/ Largest Winning Trade":"",
        "Min Return/ Largest Loosing Trade": "",
        "Risk Reward": "Max Return/Min Return",
        "Net Profit %": "sum(win_trades)+sum(loose_trades))/avg_index_val*100",
        "Sharpe Ratio": ""
    }

    if len(parameters) != len(definitions):
        print("\nPARAMETERS DICTIONARY AND DEFINITION DICTIONARY NOT MATCHING, PLEASE CHECK!\n")

    row_num = 10
    for key in parameters.keys():
        col_f, col_g, col_h = f"f{row_num}", f"g{row_num}", f"h{row_num}"
        ws[col_f] = key
        ws[col_g] = parameters[key]
        ws[col_h] = definitions[key]
        row_num += 1 

    # save list of trades in excel
    # df.to_excel("All Trades.xlsx", index = False, sheet_name = str(year))
    with pd.ExcelWriter("All Trades.xlsx") as writer:
        df.to_excel(writer, sheet_name=str(year), startcol = 10)
    wb.save("Strategy 1.xlsx")

In [65]:
def execute_trade(nse_data):

    buying_mode = True
    df = pd.DataFrame(columns= 
                      ["Trade Number","Buy Date","Buy Time","Buy Price", "Sell Date","Sell Time","Sell Price","Return"])
    all_returns = []
    all_close_values = []
    trades = {"Trade Number":[], "Buy Date":[] ,"Buy Time":[], "Buy Price":[], 
              "Sell Date":[], "Sell Time":[], "Sell Price":[], "Return":[]}
    year = nse_data.iloc[0]["Date"].year
    trade_number = 1
    counter = 0
    
    for index in range(len(nse_data)):
        if buying_mode == True:    
            if nse_data["Time"][index] == dt.time(9,15,0):
                counter = 0

            if nse_data["Close"][index] >= nse_data["SMA50"][index]:
                counter += 1
            else:
                counter = -1

            if nse_data["Time"][index] == dt.time(15,25,0):
                if counter == 75:
                    # all 75 ticks satisfied the condition
                    # trigger buy
                    buy_price = nse_data["Close"][index]
                    trades["Trade Number"].append(trade_number)
                    trades["Buy Date"].append(nse_data["Date"][index])
                    trades["Buy Time"].append(nse_data["Time"][index])
                    trades["Buy Price"].append(nse_data["Close"][index])
                    buying_mode = False
                    continue

        if buying_mode == False:
            if nse_data["Close"][index] < nse_data["SMA50"][index]:
                selling_price = nse_data["Close"][index]
                return_made = selling_price - buy_price
                trade_number += 1
                all_returns.append(return_made)
                all_close_values.extend([buy_price, selling_price])
                trades["Sell Date"].append(nse_data["Date"][index])
                trades["Sell Time"].append(nse_data["Time"][index])
                trades["Sell Price"].append(nse_data["Close"][index])
                trades["Return"].append(return_made)
                buying_mode = True
                df = pd.DataFrame(trades)
                
    return all_returns, all_close_values, df

In [13]:
def divide_dataset_to_years(nse_data):
    """
    List contains start index of data for each year.
    Last value contains last index in the dataset.
    """
    
    current_index, current_year = 0, nse_data.iloc[0]["Date"].year
    year_start_index_number = [current_index]

    for index, row in nse_data.iterrows():
        if row["Date"].year > current_year:
            current_year = row["Date"].year
            year_start_index_number.append(index)
    year_start_index_number.append(len(nse_data)-1)
    print(year_start_index_number)
    
    return year_start_index_number

In [36]:
def call_execute_trade(nse_data, year_start_index_number, wb):

    for i in range(len(year_start_index_number)-1):
        start_index = year_start_index_number[i]
        end_index = year_start_index_number[i+1]
        year = nse_data.iloc[start_index]["Date"].year        

        print(f"Processing for year: {year}")
        yearly_data = nse_data.iloc[start_index:end_index]
        yearly_data.reset_index(drop=True, inplace=True)
        all_returns, all_close_values, df = execute_trade(yearly_data)
        fill_excel(all_returns, year, all_close_values, df, wb)
        print(f"Completed execute_trade for year: {year}")

    # also call on summary
    print(f"Processing for Entire Data")
    all_returns, all_close_values, df = execute_trade(nse_data)
    fill_excel(all_returns, "Summary", all_close_values, df, wb)
    print(f"Completed execute_trade for entire data.")

## Execute the Progam 

In [15]:
wb = openpyxl.Workbook()
ws = wb.active
ws.title = "Summary"

ws["A1"].value = """
                    This excel sheet contains output data gathered from backtesting strategy 1
                """
wb.save('Strategy 1.xlsx')

In [16]:
year_start_index_number = divide_dataset_to_years(nse_data)

[0, 7551, 26168, 44809, 63002, 80099, 98549, 117072, 135447, 153745, 172570, 191033, 192232]


In [66]:
call_execute_trade(nse_data, year_start_index_number, wb)

Processing for year: 2011
Completed execute_trade for year: 2011
Processing for year: 2012
Completed execute_trade for year: 2012
Processing for year: 2013
Completed execute_trade for year: 2013
Processing for year: 2014
Completed execute_trade for year: 2014
Processing for year: 2015
Completed execute_trade for year: 2015
Processing for year: 2016
Completed execute_trade for year: 2016
Processing for year: 2017
Completed execute_trade for year: 2017
Processing for year: 2018
Completed execute_trade for year: 2018
Processing for year: 2019
Completed execute_trade for year: 2019
Processing for year: 2020
Completed execute_trade for year: 2020
Processing for year: 2021
Completed execute_trade for year: 2021
Processing for year: 2022
Completed execute_trade for year: 2022
Processing for Entire Data
Completed execute_trade for entire data.


## Check Yearly Data

In [32]:
i = 2
start_index = year_start_index_number[i]
end_index = year_start_index_number[i+1]
yearly_data = nse_data.iloc[start_index:end_index]
yearly_data.reset_index(drop=True, inplace=True)
yearly_data

,Date,Time,Open,High,Low,Close,SMA20,SMA50,SMA 100
0,2013-01-01,09:15:00,12549.95,12566.35,12547.15,12551.25,12473.5650,12471.488,12458.770297
1,2013-01-01,09:20:00,12551.25,12568.45,12549.35,12567.70,12478.8075,12473.226,12460.142574
2,2013-01-01,09:25:00,12567.70,12585.65,12564.95,12585.65,12484.9400,12475.492,12461.759406
3,2013-01-01,09:30:00,12585.65,12595.25,12584.90,12589.60,12491.1875,12477.770,12463.394059
4,2013-01-01,09:35:00,12589.60,12594.10,12583.10,12584.10,12497.2650,12479.993,12464.983168
...,...,...,...,...,...,...,...,...,...
18636,2013-12-31,15:05:00,11369.85,11378.00,11366.60,11374.35,11361.9350,11359.861,11365.366337
18637,2013-12-31,15:10:00,11374.35,11385.40,11374.35,11383.10,11362.4525,11360.294,11365.364356
18638,2013-12-31,15:15:00,11383.40,11396.75,11383.40,11385.55,11363.4400,11360.768,11365.331683
18639,2013-12-31,15:20:00,11385.55,11402.10,11385.55,11398.00,11365.3125,11361.413,11365.533168
